In [1]:
import phrase_entity_extraction as ph

In [2]:
ph.seq.look_ahead([['ADR_HEAD']])

uuid: input
n_init: <node: (*),(*)>:
active values: []
next values: []

In [3]:
ph.return_max_address(ph.seq, "123 main st")

'123 MAIN ST'

In [4]:
ph.get_markers(ph.seq, "123 main st po box 456 suite 100", ['ADDRESS', 'POBOX', 'SUITE'])


[[0, 1, 1, ['SUITE'], '123'],
 [0, 2, 2, ['ADDRESS'], '123 main'],
 [0, 3, 3, ['ADDRESS'], '123 main st'],
 [3, 6, 3, ['POBOX'], 'po box 456'],
 [4, 6, 2, ['POBOX'], 'box 456'],
 [5, 6, 1, ['SUITE'], '456'],
 [6, 8, 2, ['SUITE'], 'suite 100'],
 [7, 8, 1, ['SUITE'], '100']]

In [5]:
def get_interpretation(seq, arr_words, arr_search_keys):
    """INPUT: ['123', 'main']"""
    t1_encoded = ph.encode_from_word_list(arr_words)
    #print("encoded: " ,t1_encoded)
    seq.look_ahead(t1_encoded)
    next_values = seq.get_next_values()
    #print("returning ", next_values)
    return [search_key for search_key in arr_search_keys if search_key in next_values]

#get_interpretation(seq, w("123 martin luther king jr"))
get_interpretation(ph.seq, ph.w("suite 100"), ['ADDRESS', 'POBOX', 'SUITE'])

['SUITE']

In [9]:
def decompose_into_dictionary_words(domain, _seq, types):
    last_length = [-1] * len(domain)
    last_interp = ['']*len(domain)
    print("Length Domain: ", len(domain))
    for i in range(len(domain)):
        print("INPUT TO GET_INTERPRETATION: ", domain[:i + 1])
        inter1 = get_interpretation(_seq, domain[:i + 1], types)
        if any([ret_type in types for ret_type in inter1]):
            print("HIT A")
            last_interp[i] = [hit for hit in inter1 if hit in types]
            last_length[i] = i + 1
            
        if last_length[i] == -1:
            for j in range(i):
                inter2 = get_interpretation(_seq, domain[j + 1:i + 1], types)
                print("SUB INPUT for INTERPRET: ", domain[j+1:i+1], inter2, last_length[j])
                #if last_length[j] != -1 and any([ret_type in types for ret_type in inter2]):
                if any([ret_type in types for ret_type in inter2]):
                    print('HIT B')
                    last_interp[i] = [hit for hit in inter2 if hit in types]
                    last_length[i] = i - j
                    break
        #print("last_length: ", last_length)
        #print("last_interp: ", last_interp)
    #return last_length, last_interp
    print("last_length: ", last_length)
    print("last_interp: ", last_interp)
    print("BEGIN DECOMPOSITON PHASE")
    decompositions = []
    if last_length[-1] != -1:
        idx = len(domain) - 1
        while idx >= 0:
            print("idx: ", idx)
            decompositions.append(last_interp[idx]) #domain[idx + 1 - last_length[idx]:idx + 1])
            if last_length[idx] == -1:
                idx -= 1
                continue
            idx -= last_length[idx]
        decompositions = decompositions[::-1]
    print(decompositions)
    return last_length, last_interp, decompositions

decompose_into_dictionary_words(ph.w("SUITE 100 11845 W OLYMPIC BOULEVARD junk".lower()), ph.seq, ['ADDRESS', 'POBOX', 'SUITE', 'ATTN'])

Length Domain:  7
INPUT TO GET_INTERPRETATION:  ['suite']
INPUT TO GET_INTERPRETATION:  ['suite', '100']
HIT A
INPUT TO GET_INTERPRETATION:  ['suite', '100', '11845']
HIT A
INPUT TO GET_INTERPRETATION:  ['suite', '100', '11845', 'w']
SUB INPUT for INTERPRET:  ['100', '11845', 'w'] [] -1
SUB INPUT for INTERPRET:  ['11845', 'w'] ['SUITE'] 2
HIT B
INPUT TO GET_INTERPRETATION:  ['suite', '100', '11845', 'w', 'olympic']
SUB INPUT for INTERPRET:  ['100', '11845', 'w', 'olympic'] [] -1
SUB INPUT for INTERPRET:  ['11845', 'w', 'olympic'] ['ADDRESS'] 2
HIT B
INPUT TO GET_INTERPRETATION:  ['suite', '100', '11845', 'w', 'olympic', 'boulevard']
SUB INPUT for INTERPRET:  ['100', '11845', 'w', 'olympic', 'boulevard'] [] -1
SUB INPUT for INTERPRET:  ['11845', 'w', 'olympic', 'boulevard'] ['ADDRESS'] 2
HIT B
INPUT TO GET_INTERPRETATION:  ['suite', '100', '11845', 'w', 'olympic', 'boulevard', 'junk']
SUB INPUT for INTERPRET:  ['100', '11845', 'w', 'olympic', 'boulevard', 'junk'] [] -1
SUB INPUT for INT

([-1, 2, 3, 2, 3, 4, -1],
 ['', ['SUITE'], ['SUITE'], ['SUITE'], ['ADDRESS'], ['ADDRESS'], ''],
 [])

In [8]:
test_string = "DRUMS SUITE 100 11845 W OLYMPIC BOULEVARD"
decompose_into_dictionary_words(ph.w(test_string.lower().strip()), ph.seq, ['ADDRESS', 'POBOX', 'SUITE', 'ATTN'])

Length Domain:  7
INPUT TO GET_INTERPRETATION:  ['drums']
INPUT TO GET_INTERPRETATION:  ['drums', 'suite']
SUB INPUT for INTERPRET:  ['suite'] [] -1
INPUT TO GET_INTERPRETATION:  ['drums', 'suite', '100']
SUB INPUT for INTERPRET:  ['suite', '100'] ['SUITE'] -1
HIT B
INPUT TO GET_INTERPRETATION:  ['drums', 'suite', '100', '11845']
SUB INPUT for INTERPRET:  ['suite', '100', '11845'] ['SUITE'] -1
HIT B
INPUT TO GET_INTERPRETATION:  ['drums', 'suite', '100', '11845', 'w']
SUB INPUT for INTERPRET:  ['suite', '100', '11845', 'w'] [] -1
SUB INPUT for INTERPRET:  ['100', '11845', 'w'] [] -1
SUB INPUT for INTERPRET:  ['11845', 'w'] ['SUITE'] 2
HIT B
INPUT TO GET_INTERPRETATION:  ['drums', 'suite', '100', '11845', 'w', 'olympic']
SUB INPUT for INTERPRET:  ['suite', '100', '11845', 'w', 'olympic'] [] -1
SUB INPUT for INTERPRET:  ['100', '11845', 'w', 'olympic'] [] -1
SUB INPUT for INTERPRET:  ['11845', 'w', 'olympic'] ['ADDRESS'] 2
HIT B
INPUT TO GET_INTERPRETATION:  ['drums', 'suite', '100', '11

([-1, -1, 2, 3, 2, 3, 4],
 ['', '', ['SUITE'], ['SUITE'], ['SUITE'], ['ADDRESS'], ['ADDRESS']],
 ['', ['SUITE'], ['ADDRESS']])